In [ ]:
#import libraries
import tensorflow as tf
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
#from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from keras.models import save_model
import cv2

In [ ]:
#load dataset
zip_obj = zipfile.ZipFile(file = '/content/drive/MyDrive/Computer Vision Masterclass/Datasets/homer_bart_2.zip', mode = 'r')    #create ZipFile object in read mode
zip_obj.extractall('./')    #extract in current location
zip_obj.close() #close to release memory
#tf.keras.preprocessing.image.load_img('homer_bart_2/training_set/homer/homer100.bmp')
#tf.keras.preprocessing.image.load_img('homer_bart_2/training_set/bart/bart100.bmp')

In [ ]:
#train test split
#data augmentation
train_generator = ImageDataGenerator(rescale = 1./255,  #generate train images
                                     rotation_range=7,
                                     horizontal_flip=True,
                                     zoom_range=0.2)

train_dataset = train_generator.flow_from_directory('homer_bart_2/training_set',
                                                    target_size=(256, 256),
                                                    batch_size = 8,
                                                    class_mode='categorical',
                                                    shuffle=True)


In [ ]:
test_generator = ImageDataGenerator(rescale = 1./255)

test_dataset = test_generator.flow_from_directory('homer_bart_2/test_set', 
                                                  target_size=(256,256),
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
print(train_dataset.classes, train_dataset.class_indices,sep='\n\n')

In [ ]:
#transfer learning
base_model = tf.keras.applications.ResNet50(weights='imagenet',
                                            include_top=False,
                                            input_tensor=Input(shape=(256, 256,3)))
#train using imagenet dataset
#include_top = False - only bottom layers ae included
#input_tensor - shape of input the model should get (since we should send our image
#dataset we should specify the sizes (many sizes are available ...refer documentation))
base_model.trainable=False
#since we need to use weights and so not needed to train again

In [ ]:
#visualise model
base_model.summary()
#plot_model(base_model,'ResNet50_Model.jpg', show_shapes=True)
print("Number of layers in base_model :",len(base_model.layers))
print(base_model.output)    #return last layer

In [ ]:
#Custom dense layers
#(2048+2)/2 = 1025 hidden layer neurons
head_model = base_model.output      #joins last layer of base model with head model
head_model = GlobalAveragePooling2D()(head_model)       #other way to flatten and connect with head_model
head_model = Dense(1025, activation='relu')(head_model) 
head_model = Dropout(0.2)(head_model)
head_model = Dense(1025, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(2, activation='softmax')(head_model)

In [ ]:
#combine base_model with head_model
model = Model(inputs = base_model.input, outputs = head_model)

In [ ]:
#visualise entire model
model.summary()
#plot_model(model,'Entire_Model.jpg', show_shapes=True)

In [ ]:
#compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train
model_history = model.fit(train_dataset, epochs = 50)

In [ ]:
#evaluate
predictions = model.predict(test_dataset)
predictions = np.argmax(predictions, axis =1)
print("Accuracy :",accuracy_score(predictions, test_dataset.classes)*100)
#even transfer learning gives lesser accuracy than our own custom CNN
#because might be due to that imagenet dataset is real world while 
#this dataset is animated
cm = confusion_matrix(predictions, test_dataset.classes)
sns.heatmap(cm, annot=True)
print(classification_report(predictions, test_dataset.classes))

In [ ]:
#fine tuning
for layer in base_model.layers:
    print(layer, layer.trainable)   #view trainability of all layers
    
print(len(base_model.layers))   #find how many layers totally (175)

for layer in base_model.layers[140:]:
    layer.trainable = True          #setting layers after 140 to be trainable

In [ ]:
for layer in base_model.layers:
    print(layer, layer.trainable)   #view trainability of all layers

In [ ]:
#compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train
model_history = model.fit(train_dataset, epochs = 50)

In [ ]:
#evaluation on test set
predictions = model.predict(test_dataset)
#since we used categorical model(not binary) we get probabilities for an image to be in each class
#so we consider that the image belogs to the class with higher probability
#print(predictions)
predictions = np.argmax(predictions, axis=1)
#print(test_dataset.classes) #expected classes
#print('\n',predictions) #predicted classes
print("Accuracy :",accuracy_score(predictions, test_dataset.classes)*100)
cm = confusion_matrix(predictions, test_dataset.classes)
sns.heatmap(cm, annot=True)
print(classification_report(predictions, test_dataset.classes))

In [ ]:
#save model
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model_saved = save_model(model, './weights.hdf5')

In [ ]:
#load model
with open('model.json', 'r') as json_file:
    json_saved_model = json_file.read()

model_loaded = tf.keras.models.model_from_json(json_saved_model)
model_loaded.load_weights('weights.hdf5')
model_loaded.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_loaded.summary()

In [ ]:
#test single image
path = 'homer_bart_2/test_set/homer/homer16.bmp'
image = cv2.imread(path)
image = cv2.resize(image, (256,256))
image = image/ 255    #normalise
#print(image.shape)
image = image.reshape(-1, 256,256,3)      #reshape in format to send more than one image to predict
#print(image.shape)

result = model_loaded(image)
#print(result)  #probabilities that the image belong to each class
result = np.argmax(result, axis=1)

if(result==0):
    print('Bart')
else:
    print('Homer')

In [ ]:
tf.keras.preprocessing.image.load_img(path)